In [0]:
import os
import numpy as np
import pandas as pd 
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import seaborn
from sklearn.utils import shuffle

In [0]:
train = pd.read_csv('train.csv')
Y = train.is_promoted

In [0]:
test = pd.read_csv('test.csv')
test_ID = test.employee_id

In [46]:
print('Train shape', train.shape)
print('Test shape',  test.shape)

Train shape (54808, 14)
Test shape (23490, 13)


In [0]:
train.head()

In [0]:
test.head()

In [0]:
train.isnull().sum(axis=1).head(15)

In [0]:
train.isnull().sum(axis=0).head(15)

In [0]:
test.isnull().sum(axis=1).head(15)

In [0]:
test.isnull().sum(axis=0).head(15)

In [0]:
feats_counts = train.nunique(dropna = False)

In [0]:
feats_counts.sort_values()[:]

In [0]:
nunique = test.nunique(dropna=False).sort_values()
nunique

In [47]:
count_class_0, count_class_1 = train.is_promoted.value_counts()
print(count_class_0)
print(count_class_1 )

50140
4668


In [49]:
df_class_1 = train[train['is_promoted'] == 1]
train = pd.concat([train, df_class_1], axis = 0)
count_class_0, count_class_1 = train.is_promoted.value_counts()
print(count_class_0)
print(count_class_1 )

50140
18672


In [0]:
plt.figure(figsize=(14,6))
_ = plt.hist(nunique.astype(float)/train.shape[0], bins=100)

In [0]:
train.replace('NaN', -999, inplace=True)
test.replace('NaN', -999, inplace=True)

In [0]:
train.head()

In [0]:
test.head()

In [0]:
cat_cols = list(train.select_dtypes(include=['object']).columns)
num_cols = list(train.select_dtypes(exclude=['object']).columns)
print(cat_cols)
print(num_cols)

# ENCODING and Training

In [0]:
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score ,f1_score
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [52]:
#train = shuffle(train)
traintest = pd.concat([train.drop(['is_promoted','recruitment_channel'],axis = 1)
                       , test.drop(['recruitment_channel'],axis = 1)], axis = 0)
print(traintest.shape)
X_tt = traintest.copy()

y = train.is_promoted
#X = X.drop('is_promoted', axis=1)
X_tt = X_tt.drop('employee_id', axis=1)

print(X_tt.shape)
X = X_tt.fillna(X_tt.mean())
print(X.shape)

##Label-encoding
for c in traintest.columns[traintest.dtypes == 'object']:
    X[c] = X[c].factorize()[0]

scaler = StandardScaler()
print(X.shape)


(92302, 12)
(92302, 11)
(92302, 11)
(92302, 11)


In [0]:
#scaler.fit(train_x)
#train_x = scaler.transform(train_x)
#test_x = scaler.transform(test_x)
#submit_x = scaler.transform(X[54808:])
train_x, test_x, train_y, test_y = train_test_split(X[:68812], y, train_size=0.7)
submit_x = X[68812:]

In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))
import random
import time
import warnings
warnings.filterwarnings('ignore')
from subprocess import check_output

Python version: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
pandas version: 0.22.0
matplotlib version: 2.1.2
NumPy version: 1.14.5
SciPy version: 0.19.1
IPython version: 5.5.0
scikit-learn version: 0.19.2


In [0]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [64]:
print("Train_x Shape :: ", train_x.shape)
print("Train_y Shape :: ", train_y.shape)
print("Test_x Shape :: ", test_x.shape)
print("Test_y Shape :: ", test_y.shape)    
#Machine Learning Algorithm (MLA) Selection and Initialization
MLA = [
    #Ensemble Methods
    #ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    #ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    #gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    #linear_model.LogisticRegressionCV(),
    #linear_model.PassiveAggressiveClassifier(),
    #linear_model.RidgeClassifierCV(),
    #linear_model.SGDClassifier(),
    #linear_model.Perceptron(),
    
    #Navies Bayes
    #naive_bayes.BernoulliNB(),
    #naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    #neighbors.KNeighborsClassifier(),
    
    #SVM
    #svm.SVC(probability=True),
    #svm.NuSVC(probability=True),
    #svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    #tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    #discriminant_analysis.LinearDiscriminantAnalysis(),
    #discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]
MLA_columns = ['MLA Name','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean',  'MLA_F1_Score']
MLA_compare = pd.DataFrame(columns = MLA_columns)

row_index = 0

Train_x Shape ::  (48168, 11)
Train_y Shape ::  (48168,)
Test_x Shape ::  (20644, 11)
Test_y Shape ::  (20644,)


In [65]:
for model in MLA:

    #set name and parameters
    MLA_name = model.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    model.fit(train_x, train_y)
    predictions = model.predict(test_x)
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = accuracy_score(train_y, model.predict(train_x))
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = accuracy_score(test_y, predictions)
    MLA_compare.loc[row_index, 'MLA_F1_Score'] = f1_score(test_y, predictions)

    #save MLA predictions - see section 6 for usage
    
    #MLA_predict[MLA_name] = alg.predict(data1[data1_x_bin])
    
    row_index+=1
MLA_compare

,MLA Name,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA_F1_Score
0,BaggingClassifier,0.997986,0.961781,0.932024
1,GradientBoostingClassifier,0.832275,0.833123,0.596084
2,RandomForestClassifier,0.998588,0.970064,0.94628
3,DecisionTreeClassifier,0.999543,0.942937,0.903221
4,XGBClassifier,0.828932,0.829587,0.58641


In [69]:
estimators= []
estimators.append(('bdtc1',ensemble.BaggingClassifier()))
estimators.append(('xgb1',tree.DecisionTreeClassifier()))
estimators.append(('gbc1',XGBClassifier()    ))
estimators.append(('nn', ensemble.GradientBoostingClassifier()))
estimators.append(('nn1',ensemble.RandomForestClassifier(),))

model = VotingClassifier(estimators)
model.fit(train_x,train_y)
predictions = model.predict(test_x)
# Train and Test Accuracy
print(classification_report(test_y,predictions))
print("Train Accuracy :: ", accuracy_score(train_y, model.predict(train_x)))
print("Test Accuracy  :: ", accuracy_score(test_y, predictions))
print("F1_SCORE       :: ", f1_score(test_y, predictions))


             precision    recall  f1-score   support

          0       0.99      0.98      0.98     15105
          1       0.93      0.96      0.95      5539

avg / total       0.97      0.97      0.97     20644

Train Accuracy ::  0.9980277362564358
Test Accuracy  ::  0.9719046696376671
F1_SCORE       ::  0.9484444444444444


In [70]:
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA_F1_Score'], ascending = False, inplace = True)
MLA_compare
#MLA_predict

,MLA Name,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA_F1_Score
2,RandomForestClassifier,0.998588,0.970064,0.94628
0,BaggingClassifier,0.997986,0.961781,0.932024
3,DecisionTreeClassifier,0.999543,0.942937,0.903221
1,GradientBoostingClassifier,0.832275,0.833123,0.596084
4,XGBClassifier,0.828932,0.829587,0.58641


In [68]:
print("Train_x Shape :: ", train_x.shape)
print("Train_y Shape :: ", train_y.shape)
print("Test_x Shape :: ", test_x.shape)
print("Test_y Shape :: ", test_y.shape)    
#rf = XGBClassifier()
#rf = GradientBoostingClassifier(n_estimators=30, random_state=7)
#rf.fit(train_x,train_y)
cart0 = DecisionTreeClassifier()
cart1 = RandomForestClassifier()
num_trees = 27
model1 = KNeighborsClassifier(n_neighbors=3)
model0 = BaggingClassifier(base_estimator=cart0, n_estimators=num_trees, random_state=7)  #num_trees = 27 # .501 
model = MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=1000)
model2 = XGBClassifier(max_depth=10,min_child_weight=1,learning_rate=0.01,n_estimators=500,objective='binary:logistic')                                                                                  # .45
model3 = GradientBoostingClassifier(n_estimators=22,max_depth=10)                                          # .48
estimators= []
estimators.append(('bdtc1',model0))
estimators.append(('xgb1',model2))
estimators.append(('gbc1',model3))
estimators.append(('nn', model1))
#model = VotingClassifier(estimators)


model.fit(train_x,train_y)
predictions = model.predict(test_x)
# Train and Test Accuracy
print(classification_report(test_y,predictions))

Train_x Shape ::  (48168, 11)
Train_y Shape ::  (48168,)
Test_x Shape ::  (20644, 11)
Test_y Shape ::  (20644,)
             precision    recall  f1-score   support

          0       0.84      0.91      0.88     15105
          1       0.69      0.53      0.60      5539

avg / total       0.80      0.81      0.80     20644



In [71]:
print("Train Accuracy :: ", accuracy_score(train_y, model.predict(train_x)))
print("Test Accuracy  :: ", accuracy_score(test_y, predictions))
print("F1_SCORE       :: ", f1_score(test_y, predictions))

Train Accuracy ::  0.9980277362564358
Test Accuracy  ::  0.9719046696376671
F1_SCORE       ::  0.9484444444444444


In [72]:
y = test.employee_id
predictions = model.predict(submit_x)
df =pd.DataFrame({'employee_id' : [], 'is_promoted' : []})
print(len(X))
print(len(y))
print(len(predictions))

92302
23490
23490


In [0]:
df['employee_id'] = y
df['is_promoted'] = predictions
df.to_csv('ans.csv',index=False)

In [74]:
train['is_promoted'].value_counts()


0    50140
1    18672
Name: is_promoted, dtype: int64

In [75]:
df['is_promoted'].value_counts()

0    22145
1     1345
Name: is_promoted, dtype: int64